# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

#### 1. Read `tic-tac-toe.csv` into a dataframe.

In [53]:
import logging
logging.getLogger('tensorflow').disabled = True

In [54]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [55]:
df = pd.read_csv('tic-tac-toe.csv')

In [56]:
y = df['class'].astype(object)
X = df.drop(['class'], axis = 1)

In [57]:
y.dtype

dtype('O')

In [58]:
X

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,x,x,x,x,o,o,x,o,o
1,x,x,x,x,o,o,o,x,o
2,x,x,x,x,o,o,o,o,x
3,x,x,x,x,o,o,o,b,b
4,x,x,x,x,o,o,b,o,b
...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x
954,o,x,o,x,x,o,x,o,x
955,o,x,o,x,o,x,x,o,x
956,o,x,o,o,x,x,x,o,x


#### 2. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.


In [59]:
print(df.columns)
len(df.columns)

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')


10

In [60]:
num_victories = len(df[df['class']==True])
num_defeats = len(df[df['class']==False])

print("Number of victories of player X: {}".format(num_victories))
print("Number of victories of player O: {}".format(num_defeats))

Number of victories of player X: 626
Number of victories of player O: 332


We can see that there are no null values

In [61]:
df.isna().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

#### 3. Convert the categorical values to numeric in all columns.


Checking out the data types. Everything seems quite right, let's convert them.

In [62]:
df.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [63]:
for column in X.columns:
    X[column] = X[column].map({"x":1, "o":0, "b":0.5})

In [64]:
X

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.5,0.5
4,1.0,1.0,1.0,1.0,0.0,0.0,0.5,0.0,0.5
...,...,...,...,...,...,...,...,...,...
953,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
954,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
955,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
956,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0


In [65]:
X.dtypes

TL    float64
TM    float64
TR    float64
ML    float64
MM    float64
MR    float64
BL    float64
BM    float64
BR    float64
dtype: object

#### 4. Separate the inputs and output.


In [66]:
X, y

(      TL   TM   TR   ML   MM   MR   BL   BM   BR
 0    1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
 1    1.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0
 2    1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0
 3    1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.5  0.5
 4    1.0  1.0  1.0  1.0  0.0  0.0  0.5  0.0  0.5
 ..   ...  ...  ...  ...  ...  ...  ...  ...  ...
 953  0.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0
 954  0.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0
 955  0.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0  1.0
 956  0.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0
 957  0.0  0.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0
 
 [958 rows x 9 columns],
 0       True
 1       True
 2       True
 3       True
 4       True
        ...  
 953    False
 954    False
 955    False
 956    False
 957    False
 Name: class, Length: 958, dtype: object)

Encoding the target

In [67]:
type(y)

pandas.core.series.Series

In [68]:
y = y.map({True:1, False:0})

In [69]:
y

0      1
1      1
2      1
3      1
4      1
      ..
953    0
954    0
955    0
956    0
957    0
Name: class, Length: 958, dtype: int64

#### 5. Normalize the input data.

I have chosen 1 for player 1, 0 for player 2 and 0.5 for not chosen place

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [70]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


#I'm using sklearn for NN <- this is not the best implementation, you should use keras instead
from sklearn.datasets import fetch_openml
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier

#### 1. Split the training and test data.

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#### 2. Create a `Sequential` model.

In [73]:
pd.DataFrame(X_train)

,TL,TM,TR,ML,MM,MR,BL,BM,BR
302,0.0,1.0,1.0,0.0,1.0,0.0,0.5,1.0,0.5
467,0.0,0.5,0.0,0.0,1.0,0.5,1.0,1.0,1.0
294,1.0,0.5,0.5,0.5,1.0,0.5,0.0,0.0,1.0
548,0.5,0.0,1.0,0.0,0.5,1.0,0.5,0.5,1.0
465,0.0,0.5,0.0,1.0,0.0,0.5,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
106,1.0,1.0,0.5,0.0,1.0,0.0,0.0,0.5,1.0
270,1.0,0.5,0.0,0.5,1.0,0.0,0.0,1.0,1.0
860,0.0,0.5,1.0,0.0,1.0,0.5,0.0,0.5,1.0
435,0.0,0.5,1.0,0.0,1.0,0.0,1.0,0.5,1.0


#### 3. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.

In [74]:
from tensorflow.python import tf2
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [75]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(64, activation='relu', input_shape = (9,)), # 9 = 3*3 dense networks means all neurons in one layer are connected to all neurons of the next layer
    Dense(64, activation='relu'),    # choosing relu instead of sigmoid, this is somewhat common
    Dense(1, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

#### 4. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.

In [76]:
# Compile the model.
model.compile(
    optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
    loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
    metrics=['accuracy'],
)

#### 5. Fit the training data.

In [77]:
X_train.shape, y_train.shape

((766, 9), (766,))

In [78]:
# Train the model.
history = model.fit(
    X_train,
    y_train, # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
    epochs = 20, # go 20 times through the whole dataset
    batch_size = 32, # send 32 images at a time before you tweak the network again, to make it faster
)

Epoch 1/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 2/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 3/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 4/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 5/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 6/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 7/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 8/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 9/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 10/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+0

#### 6. Evaluate your neural network model with the test data.

In [79]:
# Evaluate the model.
model.evaluate(
    X_test,
    y_test
)

6/6 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6510


[0.0, 0.6510416865348816]

#### 7. Save your model as `tic-tac-toe.model`.

In [80]:
!mkdir -p saved_model
model.save('tic-tac-toe.model')

In [81]:
# new_model = tf2.keras.models.load_model('tic-tac-toe.model')

Vuelve a cargar un modelo Keras nuevo desde el modelo guardado:

#### I am going to compare the results with logistic regression models

In [82]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='saga').fit(X_train, y_train)

In [83]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [84]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.9660574412532638
0.9427083333333334


In [85]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [86]:
y_train_pred[:5], y_test_pred[:5]

(array([1, 1, 1, 1, 1]), array([1, 1, 1, 0, 0]))

As we can see, the model is able to predict who is going to win that round

#### R2 score

In [87]:
from sklearn.metrics import r2_score
print("Score for the train set: ", r2_score(y_train, y_train_pred))
print("Score for the test set: ", r2_score(y_test, y_test_pred))

Score for the train set:  0.8499905848679998
Score for the test set:  0.7478208955223881


In [105]:
print(y_train_pred[:10], "\n",y_test_pred[:10])

[1 1 1 1 1 1 1 0 1 1] 
 [1 1 1 0 0 1 1 1 1 0]


#### MSE

In [90]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
mse = mean_squared_error(y_test,y_test_pred)
mse

0.057291666666666664

There is no correct value for MSE. Simply put, the lower the value the better and 0 means the model is perfect

https://www.bmc.com/blogs/mean-squared-error-r2-and-variance-in-regression-analysis/

#### RMSE

In [91]:
rmse = np.sqrt(mean_squared_error(y_test,y_test_pred))
rmse

0.23935677693908453

#### MAE

In [92]:
mae = mean_absolute_error(y_test, y_test_pred)
mae

0.057291666666666664

What is a good MAE score? <br>

The closer MAE is to 0, the more accurate the model is. But MAE is returned on the same scale as the target you are predicting for and therefore there isn’t a general rule for what a good score is. How good your score is can only be evaluated within your dataset.

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [114]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(64, activation='relu', input_shape = (9,)), # 9 = 3*3 dense networks means all neurons in one layer are connected to all neurons of the next layer
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),    # choosing relu instead of sigmoid, this is somewhat common
    Dense(1, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

In [115]:
# Compile the model.
model.compile(
    optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
    loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
    metrics=['accuracy'],
)

In [116]:
# Train the model.
history = model.fit(
    X_train,
    y_train, # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
    epochs = 20, # go 20 times through the whole dataset
    batch_size = 32, # send 32 images at a time before you tweak the network again, to make it faster
)

# Evaluate the model.
model.evaluate(
    X_test,
    y_test
)

Epoch 1/20
24/24 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 2/20
24/24 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 3/20
24/24 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 4/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 5/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 6/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 7/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 8/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 9/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6540
Epoch 10/20
24/24 [==============================] - 0s 1ms/step - loss: 0.0000e+0

[0.0, 0.6510416865348816]

**Which approach(es) did you find helpful to improve your model performance?**

Still in progress

In [94]:
# your answer here